In [ ]:
# Install OpenAI SDK
!pip install openai
import openai
from google.colab import userdata
# configure your own openai api-key and add it to google collab secrets
openai_api_key = userdata.get('openai-key')





#  What is an LLM?

A **Large Language Model (LLM)** is a very large neural network trained to predict the next word in a sentence.  
It read **billions of words** and can now:

- Complete your sentences
- Write essays or tweets
- Answer questions
- Generate code

### 🧒 Explain Like I'm 5:
> Imagine a robot that read all the books in the world. When you ask it something, it uses everything it read to guess the best words to say next.

# 💬 Multi-Turn Chat with Roles Example

This example demonstrates how you can maintain a **conversation** with the LLM using different `role` messages.

### 🧠 Roles Recap:
- `system`: Sets the overall behavior or tone of the assistant.
- `user`: User input prompts.
- `assistant`: Previous LLM responses (helps maintain context).

---
# 🧮 Tokens, Context

### 🔹 What is a Token?
- A token is like a chunk of text (e.g., a word or sub-word).
- "hackathon is cool!" → 4 tokens

###🔹 Context Window
- LLMs remember a limited number of tokens per conversation:

- GPT‑3.5‑turbo: 4 096‑token output cap (≈ short‑term memory).
- GPT‑4o‑mini: up to 128 000 tokens in one prompt + response.

When that limit is reached, the API simply drops the oldest messages, so early context vanishes.
OpenAI Platform




In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

#
messages = [
    # ▸ SYSTEM role: high‑level behavior instructions. Parsed first.
     {"role": "system",
     "content": "You are a patient science tutor who adapts explanations to a 5‑year‑old."},

    # ▸ USER role: the end‑user’s request or follow‑up.
    {"role": "user",
     "content": "Explain black holes like I'm 5"}
]

# 4) Call the new endpoint namespace openai.chat.completions.create
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

# 5) Extract the answer and token usage (handy for cost tracking)
print(response.choices[0].message.content)
print("Total tokens (prompt + completion):", response.usage.total_tokens)



# **Cost** calculation
#### gpt-4o-mini Pricing per **1M** tokens
                   
| Token type | Price (USD) |
| ---------- | ----------- |
| **Input**  | **\$0.15**  |
| **Output** | **\$0.60**  |


In [ ]:
def calculate_mini_response_cost(response):
    """
    Return the USD cost for a single GPT‑4o mini call.

    Parameters
    ----------
    response : openai.openai_object.OpenAIObject
        The object returned by openai.ChatCompletion.create()

    Returns
    -------
    float
        Total cost in USD.
    """
    PRICE_INPUT_PM  = 0.15
    PRICE_OUTPUT_PM = 0.60
    prompt_tokens     = response.usage.prompt_tokens # input tokens
    completion_tokens = response.usage.completion_tokens # response tokens

    # Compute costs
    cost_input  = (prompt_tokens     * PRICE_INPUT_PM)  / 1_000_000
    cost_output = (completion_tokens * PRICE_OUTPUT_PM) / 1_000_000

    return cost_input + cost_output

print("Total tokens (prompt + completion):", response.usage.total_tokens)
print(f"Total cost: ${calculate_mini_response_cost(response=response):,.6f}")









### 🗣️ Example Conversation Flow:
- The system tells the model to behave like a **funny tour guide **.
- The user asks about the Eiffel Tower.
- The assistant replies sarcastically .
- The user follows up, and the LLM responds again using the context.

### 🔹 Message Roles
| Role        | Use Case                                 |
|-------------|-------------------------------------------|
| `system`    | Sets behavior and tone                   |
| `user`      | Your input to the model                  |
| `assistant` | Model’s reply (or past replies)          |
| `tool`      | Where the model calls a tool/function    |

---


In [ ]:
messages = [
    {"role": "system", "content": "You are a funny tour guide."},
    {"role": "user", "content": "Tell me about the Eiffel Tower."},
    {"role": "assistant", "content": "The Eiffel Tower is like a giant iron ice cream cone! Want to know how tall it is?"},
    {"role": "user", "content": "where is this giant iron ice cream cone"}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)



## Let’s build an LLM **“Private Tutor”** app  
### A chat‑based tutor that adapts explanations, examples, and practice questions to:

* **Middle School**
  * Concrete, everyday examples  
  * Visual aids and short explanations  

* **High School**
  * Explain scientific concepts
  * Answers SAT style questions

* **University**
  * Deep‑dive theory with detailed explaination   
  * Research‑style problems and step‑by‑step proofs  

## let's explore prompting techniques
  * zero-shot , one-shot , few-shot  
  * chain of thought

In [7]:
# ----------------------------------------
# System prompts for each education level
# ----------------------------------------
system = {
    # zero shot prompting
    "middle": (
        """
        You are a **middle‑school** science tutor.
        Use simple words, fun analogies, and short sentences."""
    ),

    # one shot prompting
    "high": (
        """
        You are a **high‑school** physics tutor.
        Give clear explanations, answered SAT exams examples
         one-shot-example:
    user : What are electrons?
    assistant : Electrons are tiny, negatively‑charged sub‑atomic particles
    that orbit the nucleus of an atom.  They’re about 1⁄2000 the mass of a
    proton, and their arrangement in “shells” determines an element’s
    chemical behavior.  SAT tip: think of electrons as the mobile charge
    carriers in a circuit—when a question mentions electric current, it’s
    the flow of electrons that earns you the point.
    """
    ),

     # chain of thought prompting
    "university": (
        """
        You are a **university‐level astrophysics lecturer**.

    **Chain‑of‑Thought Requirement**
    • Think step‑by‑step, stating assumptions and intermediate deductions.
    • Write relevant equations and define each variable.
    • Do **not** hide reasoning; show the logical flow that leads to the answer.
    • If the question is ambiguous, ask one clarifying question before proceeding.

        """
    ),
}

user_query = "Explain black holes"


# Middle School Level

In [ ]:
messages_middle = [
    {"role": "system", "content": system["middle"]},
    {"role": "user", "content": user_query},
]
response_middle = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_middle
)
print("\n--- Middle School Level ---")
print(response_middle.choices[0].message.content)

# High School Level

In [ ]:
messages_high = [
    {"role": "system", "content": system["high"]},
    {"role": "user", "content": user_query},
]
response_high = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_high
)
print("\n--- High School Level ---")
print(response_high.choices[0].message.content)

# University Level

In [ ]:
messages_high = [
    {"role": "system", "content": system["university"]},
    {"role": "user", "content": user_query},
]
response_high = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_high
)
print("\n--- University Level ---")
print(response_high.choices[0].message.content)

## 🔍 Retrieval‑Augmented Generation (RAG)

RAG **retrieves** the most relevant passages from a knowledge base and **augments** the LLM’s prompt with those passages before it generates an answer.

### Why RAG matters for the Private Tutor

- **Reinforces context.** Supplies up‑to‑date facts and equations that may be missing from the model’s parameters.  
- **Reduces hallucinations.** Grounding the answer in retrieved text keeps explanations accurate and complete.  
- **Delivers key formulas on demand.** Ensures students see the exact equations they need, not a guess or omission.  


In [ ]:
#  Tiny Knowledge‑Base + RAG Helper

# can be replaced by a vector search database containing university books and formulas .
# You can use pinecone , chromadb , weviate , any vector database provider
KB = [
    "Newton’s law of universal gravitation:  F = G·m₁m₂ / r²  where  G = 6.67430 × 10⁻¹¹ N·m²·kg⁻².",

    "Einstein’s mass–energy equivalence:  E = m·c²  with  c = 2.99792458 × 10⁸ m s⁻¹.",

    "Faraday–Lenz law of induction:  ∇ × E = −∂B/∂t,  relating a changing magnetic field B to an induced electric field E.",

    "Quantum mechanics:  iħ·∂ψ/∂t = Ĥψ,  where ħ = 1.054 × 10⁻³⁴ J·s and Ĥ is the Hamiltonian operator.",

    "Chemical thermodynamics:  ΔG = ΔH − TΔS.  A reaction is spontaneous at constant T and P when ΔG < 0.",

    "Ideal gas law:  P·V = n·R·T,  with R = 8.314 J·mol⁻¹·K⁻¹.",

    "Fluid dynamics (incompressible):  ρ(∂v/∂t + v·∇v) = −∇p + μ∇²v,  linking momentum, pressure p and viscosity μ.",

    "Signal processing:  F(ω) = ∫_{−∞}^{∞} f(t)·e^{−iωt} dt  and  f(t) = (1/2π)∫_{−∞}^{∞} F(ω)·e^{iωt} dω.",

    "Black‑body radiation:  B(λ,T) = (2hc²/λ⁵)·1/(e^{hc/(λkT)} − 1),  with h = 6.626 × 10⁻³⁴ J·s.",

    "Bernoulli’s equation for inviscid flow:  p + ½ρv² + ρgh = constant along a streamline."
]


import  numpy as np
EMBED_MODEL = "text-embedding-3-small" # OPENAI embeddeing model

kb_vecs = client.embeddings.create(
    model=EMBED_MODEL,
    input=KB
)
kb_vecs = np.array([d.embedding for d in kb_vecs.data])  # kb_vecs is our mini vector database

# Simple cosine‑similarity retriever
# top_k = number of chunks retrieved
def retrieve_university_books(query: str, top_k: int = 1) -> list[str]:
    """Return the top‑k relevant chapters facts most relevant to `query`."""
    q_vec = np.array(
        client.embeddings.create(model=EMBED_MODEL, input=[query]).data[0].embedding
    )
    sims = kb_vecs @ q_vec / (np.linalg.norm(kb_vecs, axis=1) * np.linalg.norm(q_vec))
    top_ids = sims.argsort()[-top_k:][::-1]
    return [KB[i] for i in top_ids]

print("Top facts for:  What causes gravity?\n")
for formula in retrieve_university_books("What causes gravity?"):
    print("related formula : ", formula)


In [ ]:

# Reinforced by RAG
def answer_university_student(query: str):
    """
    Retrieve context from the KB and return a chain‑of‑thought answer.
    (k is fixed inside retrieve_university_books; caller doesn’t need to pass it.)
    """
    context = "\n".join(retrieve_university_books(query))

    messages = [
        {"role": "system", "content": system['university']},
        {"role": "assistant", "content": f"Reference book:\n{context}"}, # add the retrieved context
        {"role": "user", "content": query},
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return response.choices[0].message.content


print(answer_university_student("What causes gravity?"))


# 🛠️ Tool Calling Setup (Function Calling)

LLMs like GPT-4o-mini can go beyond just generating text — they can **request actions** using tools you define.

---

### 🔹 What is Tool Calling?
Instead of responding with plain text, the LLM can say:
> “I want to call a function named `XXXXX` with this input.”

Your app will:
1. Detect this tool call.
2. Run the real function (like a Python script or API call).
3. Pass the result **back to the LLM** to continue the conversation.

---

### 🔧 In This Example:
We define a **language transcribtion** named `gpt-to-egytpian` with:
- A `name`: what the LLM calls.
- A `description`: tells the LLM what it does.
- `parameters`: what input the tool needs (`query` in this case).

This tool simulates an API that searches for trending topics.
### 📨 Tool Message Type
When the LLM wants to invoke a tool, it responds with a special **`tool` message type** rather than plain text. This message includes the function name and a structured JSON input.  
Your system detects this message and triggers the corresponding tool before resuming the LLM conversation with the result.



In [ ]:
import json
# 🛠️ Tool definition
tools = [
    {
        "type": "function",
        "function": {
            "name": "gpt_to_egyptian",
            "description": "Translate English text into Egyptian‑Arabic ",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "English text"}
                },
                "required": ["text"]
            }
        }
    }
]

def gpt_to_egyptian(text: str) -> str:
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Translate to friendly Egyptian Arabic in a sarcastic manner."},
            {"role": "user", "content": text}
        ]
    )
    return r.choices[0].message.content.strip()

queries = [
    "Please translate to Arabic: 'Good morning, everyone!'",
    "What is the Pythagorean theorem?",
    "Translate to Arabic: 'I love programming in Python.'",
]

for q in queries:
    print(f"\n🟠 USER QUERY: {q}")

    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant Tutor . "
                "If the user asks to translate English into Arabic, "
                "call the function `gpt_to_egyptian` with the text. "
                "Otherwise, answer normally."
            )
        },
        {"role": "user", "content": q}
    ]

    first = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    ).choices[0]

    if first.finish_reason == "tool_calls":
        call = first.message.tool_calls[0]
        args = json.loads(call.function.arguments)
        print("🔧 TOOL CALLED with args:", args)

        # Perform translation
        arabic_text = gpt_to_egyptian(args["text"])
        print("🌐 Translation returned:", arabic_text)

        # Send tool message back to GPT
        tool_msg = {
            "role": "tool",
            "tool_call_id": call.id,
            "name": "gpt_to_egyptian",
            "content": arabic_text
        }

        final = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages + [first.message, tool_msg]
        )
        print("🟢 GPT FINAL ANSWER:", final.choices[0].message.content)

    else:
        # No tool; GPT responds directly
        print("🟢 GPT DIRECT ANSWER:", first.message.content)

Prompt Engineering  
&nbsp;&nbsp;↓  
RAG Context Retrieval  
&nbsp;&nbsp;↓  
LLM Pass #1 — Draft English Answer  
&nbsp;&nbsp;↓  
Tool‑Call Decision  
&nbsp;&nbsp;↓  
&nbsp;&nbsp;├─ Arabic requested → Translation Tool (`gpt_to_egyptian`) → Final Arabic Answer  
&nbsp;&nbsp;└─ Otherwise → Final English Answer


In [ ]:
# ----------------  Full application layer ----------------
def private_university_tutor(query: str) -> str:
  #1) RAG ( context retrieval )
    context = "\n".join(retrieve_university_books(query))
    print(f"\n🔹 QUERY: {query}")
    print(f"🔹 Retrieved context:\n{context}\n")
    #2) prompt engineering
    messages = [
        {
            "role": "system",
            "content": (
                "You are a **university‐level astrophysics lecturer "
                "Use the provided reference to craft a concise, accurate answer. "
                "Think step‑by‑step, stating assumptions and intermediate deductions."
                "Write relevant equations and define each variable."
                "Do **not** hide reasoning; show the logical flow that leads to the answer"
                "If the student explicitly requests Arabic output"
                "(e.g. 'in Arabic', 'بالعربي', 'translate to Arabic'), "
                "call the function `gpt_to_egyptian` with your English answer. "
                "Otherwise, answer in English."
            ),
        },
        {"role": "assistant", "content": f"Reference book:\n{context}"},
        {"role": "user",  "content": query},
    ]

    first = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
    ).choices[0]

    print("🔹 finish_reason:", first.finish_reason)
    print("🔹 First assistant message:\n", first.message.content, "\n")
#3) tool calling
    if first.finish_reason == "tool_calls":
        call = first.message.tool_calls[0]
        args = json.loads(call.function.arguments)
        print("🔧 Tool call detected:", call.function.name, "with", args)

        arabic = gpt_to_egyptian(args["text"])
        print("🌐 Local translation result:\n", arabic, "\n")

        print("🟢 Final assistant answer (Arabic):\n", arabic, "\n")
        return arabic

    print("🟢 Final assistant answer (no tool needed):\n", first.message.content, "\n")
    return first.message.content


private_university_tutor("What causes gravity?")

print("\n" + "-" * 70 + "\n")

private_university_tutor("Explain Newton's law of gravity in Arabic, please.")
